In [1]:
!pip install pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0a96154635aba46d45b851962668744532141444c0d1d2ab592860621dbc2f6f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import os

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
import time

In [3]:
# This function creates SparkContext and StreamingContext
def initStreamingContext():
    try:
      ssc.end()
    except:
      pass
    finally:
      spark_conf = SparkConf()\
            .setAppName("IrTest")\
            .setMaster("local[*]")
      sc = SparkContext.getOrCreate(spark_conf)
      # Creating Streaming Context with batch window size (interval time) of 1 second
      ssc = StreamingContext(sc, 1)
      return ssc

In [7]:
# Let's create ssc.
ssc = initStreamingContext()
# Initialize a DStream by connecting it to a TCP socket.
# The server will start sending data which goes to the DStream.
receivedData = ssc.socketTextStream("datasci.cs.uwaterloo.ca", 4321)
receivedData.pprint()
ssc.start()
# Just wait 5 seconds before the stream stop.
time.sleep(10)
ssc.stop()

-------------------------------------------
Time: 2024-05-27 10:45:17
-------------------------------------------

-------------------------------------------
Time: 2024-05-27 10:45:18
-------------------------------------------
0.438,0.498,3.625,3.645,5.000,2.918,5.000,2.351,2.332,2.643,1.698,1.687,1.698,1.717,1.744,0.593,0.502,0.493,0.504,0.445,0.431,0.444,0.440,0.429,Slight-Right-Turn
0.438,0.498,3.625,3.648,5.000,2.918,5.000,2.637,2.332,2.649,1.695,1.687,1.695,1.720,1.744,0.592,0.502,0.493,0.504,0.449,0.431,0.444,0.443,0.429,Slight-Right-Turn
0.438,0.498,3.625,3.629,5.000,2.918,5.000,2.637,2.334,2.643,1.696,1.687,1.695,1.717,1.744,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.446,0.429,Slight-Right-Turn
0.437,0.501,3.625,3.626,5.000,2.918,5.000,2.353,2.334,2.642,1.730,1.687,1.695,1.717,1.744,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.444,0.429,Slight-Right-Turn
0.438,0.498,3.626,3.629,5.000,2.918,5.000,2.640,2.334,2.639,1.696,1.687,1.695,1.717,1.744,0.592,0.502,0.493,0.504,0.449,0.4

In [8]:
ssc = initStreamingContext()
Data = ssc.socketTextStream("datasci.cs.uwaterloo.ca", 4321)

# flattening the sensor values, windowing back to 3 seconds and reducing to find the minimum
Data_min = Data.flatMap(lambda x: x.split(',')[:24]).window(3, 1).reduce(lambda x, y: x if x < y else y)
Data_min.pprint()

ssc.start()
# Let's wait for 20 seconds before the program is stopped.
time.sleep(20)
ssc.stop()

-------------------------------------------
Time: 2024-05-27 10:45:42
-------------------------------------------
0.429

-------------------------------------------
Time: 2024-05-27 10:45:43
-------------------------------------------
0.429

-------------------------------------------
Time: 2024-05-27 10:45:44
-------------------------------------------
0.429

-------------------------------------------
Time: 2024-05-27 10:45:45
-------------------------------------------
0.429

-------------------------------------------
Time: 2024-05-27 10:45:46
-------------------------------------------
0.444

-------------------------------------------
Time: 2024-05-27 10:45:47
-------------------------------------------
0.453



In [9]:
ssc = initStreamingContext()
Data = ssc.socketTextStream("datasci.cs.uwaterloo.ca", 4321)

# extract movements and perform windowing
Data = Data.map(lambda x: x.split(',')[24]).window(3, 1).cache()

# counting total and adding temporary key for joining
count = Data.count().map(lambda x: ('windowKey', x))

# counting by value and adding temporary key for joining
Data = Data.countByValue().map(lambda x: ('windowKey', x))

# joining to get ratios
merged = Data.join(count).map(lambda x: (x[1][0][1] / x[1][1], x[1][0][0]))

def print_rdd(rdd):
  "Custom print function for individual RDDs."
  rdd = rdd.sortByKey(False).map(lambda x: (x[1], x[0]))
  result = rdd.collect()
  print("----------")
  for record in result:
    print(record[0], record[1])

# perform action for each RDD
merged.foreachRDD(print_rdd)

ssc.start()
time.sleep(50)
ssc.stop()

----------
----------
Slight-Right-Turn 1.0
----------
Slight-Right-Turn 0.7272727272727273
Sharp-Right-Turn 0.2727272727272727
----------
Sharp-Right-Turn 0.6363636363636364
Slight-Right-Turn 0.36363636363636365
----------
Slight-Right-Turn 0.5172413793103449
Sharp-Right-Turn 0.4827586206896552
----------
Slight-Right-Turn 0.6
Sharp-Right-Turn 0.4
